In [ ]:
# Check python version
import sys
sys.version

In [13]:
!apt-get install -y libsndfile1

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libflac8 libogg0 libvorbis0a libvorbisenc2
The following NEW packages will be installed:
  libflac8 libogg0 libsndfile1 libvorbis0a libvorbisenc2
0 upgraded, 5 newly installed, 0 to remove and 17 not upgraded.
Need to get 557 kB of archives.
After this operation, 2051 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libogg0 amd64 1.3.2-1 [17.2 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libflac8 amd64 1.3.2-1 [213 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/main amd64 libvorbis0a amd64 1.3.5-4.2 [86.4 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/main amd64 libvorbisenc2 amd64 1.3.5-4.2 [70.7 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libsndfile1 amd64 1.0.28-4ubuntu0.18.04.2 [170 kB]
Fetched 557 kB in 5s (114 kB/s)     
debconf: delayi

In [16]:
# Install fastai
!pip3 install fastai kaggle soundfile

  Using cached soundfile-0.11.0-py2.py3-none-any.whl (23 kB)


# Download Dataset

In order to use the Kaggle’s public API, you must first authenticate using an API token. From the site header, click on your user profile picture, then on “My Account” from the dropdown menu. This will take you to your account settings at https://www.kaggle.com/account. Scroll down to the section of the page labelled API:

To create a new token, click on the “Create New API Token” button. This will download a fresh authentication token onto your machine.

### Accept the rules

https://www.kaggle.com/competitions/whale-detection-challenge/rules


### Upload your `kaggle.json` to the same folder as this notebook then run the cell below

In [ ]:
!mkdir -p ~/kaggle; mv kaggle.json ~/.kaggle/kaggle.json

### Download Dataset from Kaggle

In [ ]:
!kaggle competitions download -c whale-detection-challenge

### Prepare Dataset for Use

In [ ]:
!apt-get install unzip

In [5]:
!rm -rf full_data; rm -rf sample_data; rm -rf tmp_data; #remove any existing extracted data
!unzip -q whale-detection-challenge.zip -d data/ #unzip main file
!unzip -q data/small_data_sample_revised.zip -d sample_data/ #unzip sample data
!unzip -q data/whale_data.zip -d tmp_data/ #unzip full data
!rm -rf data/; rm -rf tmp_data/data/test; #remove unneeded files. official test data isn't used because we don't have labels
!mkdir full_data; mv tmp_data/data/train full_data/audio; #move stuff around
!mv tmp_data/data/train.csv full_data/labels.csv #rename labels
!rm -rf tmp_data #remove tmp directory

In [17]:
# This file contains all the main external libs we'll use
from fastai.imports import * #used for fastai
from IPython import display #used to display media in notebook
import soundfile as sf #used to load sound files
import matplotlib.pyplot as plt #used to plot in notebook
import scipy
from scipy.signal import hann
from scipy.fftpack import rfft

import pandas as pd

### Open the metadata file

Audio loading based on: https://github.com/limazix/audio-processing/blob/master/audio_classifier_fastai.ipynb

In [19]:
DATA_ROOT_DIR=os.path.normpath(os.path.join(os.getcwd(), 'full_data'))
DATA_META_FILE=os.path.join(DATA_ROOT_DIR, 'labels.csv')
DATA_AUDIO_DIR=os.path.join(DATA_ROOT_DIR, 'audio')

In [20]:
data_meta = pd.read_csv(DATA_META_FILE)
print(data_meta.iloc[0, :])

clip_name    train1.aiff
label                  0
Name: 0, dtype: object


In [22]:
def build_file_path(item):
    item['path'] = item['file_name']
    item['label'] = item['label']
    return item[['path', 'label']]

data = data_meta.apply(build_file_path, axis=1)
data.info()

KeyError: 'file_name'

In [10]:
df = pd.read_csv("./full_data/labels.csv")
df.head()

,clip_name,label
0,train1.aiff,0
1,train2.aiff,0
2,train3.aiff,0
3,train4.aiff,0
4,train5.aiff,0


In [ ]:
dls = ImageDataLoaders_from_csv(
  "./full_data/audio",
  csv_fname = "labels.csv",
  header = "infer",
  delimiter = ",",
  valid_pct = 0.2,
  seed = 42,
  fn_col = 0,
  label_col = 1,
)
path = Path("./audio")
cfg = AudioConfig.BasicMelSpectrogram(n_fft=512)
a2s = AudioToSpec.from_cfg(cfg)
auds = DataBlock(blocks = (AudioBlock, CategoryBlock),  
                 get_x = ColReader("clip_name", pref=path), 
                 valid_pct = 0.2,
                 batch_tfms = [a2s],
                 get_y = ColReader("label"))

# Overview

## Problem Description

The country depends on the shipping industry's ability to transport goods across large distances. The North Atlantic ocean houses a large portion of the nation's shipping industry as well as the last remaining 350 North Atlantic Right whales. Right whales die in large part due to collisions with commercial ships. Studies estimate that saving two female right whales per year could put their population back on track.

## Bioacoustics

Bioacoustics is the science combining biology and acoustics. In this particular domain we use bioacoustics to investigate sound as a tool to detect the presence of North Atlantic right whales.

## What does a North Atlantic Right Whale sound like?

North Atlantic right whales can make many sounds. The most distinctive of which is called the up-call. The North Atlantic right whale uses this call to tell other right whales that it is nearby. It can be thought of as a 'contact call'. It is a low frequency 'whoop' sound.

To play an audio file in Python, we first have to load that audio file into memory. As we'll see shortly, an audio file is little more than a sequence of numbers. Here's some sample Python that loads in whale call:

In [ ]:
whale_audio, whale_samplerate = sf.read("./data_sample/right_whale/train6.aiff")
display.Audio(whale_audio, rate=samplerate)

# A Brief Introduction to Signal Processing (Using Whale Sounds)

## Loading Examples

In [ ]:
whale_audio, whale_samplerate = sf.read("./data_sample/right_whale/train6.aiff")
not_whale_audio, not_whale_samplerate = sf.read("./data_sample/no_right_whale/train1.aiff")

The sampling rate refers to the number of samples of audio recorded every second. It is measured in samples per second or Hertz (abbreviated as Hz or kHz, with one kHz being 1000 Hz). 

The sampling rate is analogous to the frame rate or FPS (frames per second) measurement for videos. A video is simply a series of pictures, usually called in this context “frames”, displayed back to back very quickly to give the illusion (at least to us humans) of continuous non-interrupted motion or movement.

Source: https://www.vocitec.com/docs-tools/blog/sampling-rates-sample-depths-and-bit-rates-basic-audio-concepts

## The Raw Signal

Here we take a look at the plot of the raw signal of the audio file. Unfortunately, this representation does not provide us with very much information to analyze the contents of the audio file. The plot provides us with a representation of the signal in the time dimension as it relates to amplitude but it is not very intuitive. When we hear sound, our ears interpret pitch. Pitch is a term to describe our understanding of frequency. As humans, it may be more meaningful for us to extract to frequency content of the signal.

In [ ]:
plt.plot(whale_audio)
# label the axes
plt.ylabel("Amplitude")
plt.xlabel("Time (samples)")
# set the title
plt.title("Whale Sample")
# display the plot
plt.show()

plt.plot(not_whale_audio)
# label the axes
plt.ylabel("Amplitude")
plt.xlabel("Time (samples)")
# set the title
plt.title("Not Whale Sample")
# display the plot
plt.show()

Notice that we have a total of 4000 samples since each recording has a sample rate of 2000Hz and a total length of 2 seconds.

Lets zoom in a bit to just the first hundred samples to take a closer look a small section of time.

In [ ]:
def plot_signal(audio, title):
  plt.plot(audio)
  # label the axes
  plt.ylabel("Amplitude")
  plt.xlabel("Time (samples)")
  # set the title
  plt.title(title)
  # display the plot
  plt.xlim(0, 100)
  plt.show()

plot_signal(whale_audio, "Whale Sample Signal")
plot_signal(whale_audio, "Not Whale Sample Signal")

## Frequency Analysis

To get a clearer picture of what is going on in the audio file, it is useful to examine the frequency content of the signal. To do this we will breakdown the original signal into its frequency components and plot it. Here we perform a Fast Fourier Transform on the signal and plot it. Now, we get a clearer picture of which frequencies are used in the sound file as well as their amplitude.

In [ ]:
def plot_frequency(audio, title):
  # apply a Hanning window
  window = hann(4000)
  audio = audio * window
  # fft
  mags = abs(rfft(audio))
  # convert to dB
  mags = 20 * scipy.log10(mags)
  # normalise to 0 dB max
  mags -= max(mags)
  # plot
  plt.plot(mags)
  # label the axes
  plt.ylabel("Magnitude (dB)")
  plt.xlabel("Frequency Bin")
  # set the title
  plt.title(title)
  plt.show()

plot_frequency(whale_audio, "Whale Sound Frequency Analysis")
plot_frequency(not_whale_audio, "Not Whale Sound Frequency Analysis")

## Time-Frequency Analysis

What would be really useful would be to see which frequencies are active, and at what intensity, at a given point in time? Here we produce a spectrogram which gives us an image representing a breakdown of frequencies plotted against time. A dark color in the plot represents intense activity at that frequency at that time. This gives us a much more helpful picture where we can see in the middle the call of the whale going upwards in pitch.

In [ ]:
def show_spectrogram(audio, title):
  fs = 2000 # sampling frequency

  # Plot the spectrogram
  plt.figure(figsize=(10, 5))
  S, freqs, bins, im = plt.specgram(audio, NFFT=256, Fs=fs, noverlap=192)
  plt.xlabel('Time')
  plt.ylabel('Frequency')
  plt.title(title)
  plt.show()

show_spectrogram(whale_audio, "Whale Audio Spectrogram")
show_spectrogram(not_whale_audio, "Not Whale Audio Spectrogram")

# Cornell Lab of Ornithology Buoy System

The Cornell Lab of Ornithology used a bouy platform from the Woods Hole Oceanographic Institution. The WHOI platform components included mooring, a hydrophone, a power system, surface expression, and antennae. They also included Cornell components such as housing, analog interface hardware, GPS, an embedded computer, a detection engine, and telemetry hardware. The bouy hardware/software system would rank 2s sound clips as potential North Atlantic Right Whale candidates and would upload the clip along with meta data such as location, timestamp, and more via a satellite for shore-side processing.

## Reading a Spectogram

In [ ]:
show_spectrogram(whale_audio, "Whale Audio Spectrogram")

Recall the image above as the 'image' of what a NARW (North Atlantic Right Whale) call looks like. This image is called a spectogram and what it shows is the activity of a signal in the frequency domain plotted against time. Notice the brighter spot in the 0.5 - 1 second time range. That represents a NARW up-call as it goes up in frequency over time. The more intense the pixels in a region, the more activity in that corresponding frequency is going on at that particular time. We notice that a whale call has an upward trajectory in frequency.

## Early Detection Methods 2005-2007

Input from a hydrophone (underwater microphone) is input into an analog to digital converter. The magnitude-squared discrete fourier transform spectogram is computed from the signal.

In order to detect a NARW up-call the spectrogram is processed by Doug Gillespie's NARW upsweep detector. Essentially, an estimate for the background level of noise is formed. The detector would check from time-frequency cells with energy greater than an integer multiple of the background energy estimate. Cells with energy over this level were grouped in time and frequency (called candidate events). A numerical score from 2-10 was given to each event based on the duration of the event, min/max frequency, sweep frequency, position of min/max Hz, and max bandwidth. If the score of the event exceeded a threshold then its time and sore were stored. Low scoring events were constantly being replaced by better scoring events. After all of this only the 10 best scoring clips were to be stored in long term bouy memory. These clips would then be uploaded shore-side where experts manually annotate whether or not a whale was present in the clip.

# Kaggle Competition

In 2013, The Cornell Bioacoustics Research Program and Marinexplore held a public data science contest to come up with new solutions to NARW detection. A variety of different approaches were used by different teams. The benchmark score to beat was 0.7 ROC AUC. The method to get that score was based on the existing system in place today by Cornell, outlined above. The winning solution managed to get a score of 0.981 ROC AUC which is a significant improvement.

## Winning Approach

The winning approach meets the objective by finding an automatic way of detection NARW up-calls without the need for experts to have to sit through and manually label incoming events.

### Template Matching

#### Motivation

By looking at a large number of images of spectrograms of whale calls, patterns start to emerge. Most whale calls exist in a frequency range of 50hz-400hz. Most whale calls also include an upword trajectory in pitch. Differences exist in the slope of the trajectory and the length of the call. The winning approach uses the notion that most whale calls "look" alike, when looked at in a spectrogram, and tries to use that to it's advantage. Rather than pass an entire image (of the spectrogram) to a classification algorithm, they pass a small list of features derived from template matching (and other metrics). 

#### How does it work?

Template matching finds areas of an image that are similar to a template image. The template image is often called a patch. 

Two components are needed:
* A Source image
* Template image
    
The goal is to detect the highest matching area of a source image.

To find a matching area, you overlay the template image over the source image and compare the areas. The comparison is given a score and is stored in a result matrix R at the index (x,y) corresponding to the current rectangle formed with top left corner at location (x,y) in the source image. After the comparison is made, the method then slides the template over one pixel and compares against that rectangle. This process continues for all pixels in the source image.

A score is then calculated from this.

#### How were templates generated?

If a training sample did not correlate to any template well, then that training sample was converted to a template. This process was repeated until most training samples matched templates. This process was not automatic and was done manually. The template file read in by this method was merely a filename of an image, an x0 and xN positions (start and finish, x-axis), and a y0 and yN positions (start and finish, y-axis). Together these positions represent a rectangle inside a given image file.

##### Pre-Processing the Spectrogram Before Template Creation

Before using a given rectangle of an image as a template, they did a few pre-processing steps.

First was to demean the image.

What this does is to cut off extreme values and remove the mean at a given pixel. This enhances the contrast vertically along the frequency dimension.

The next step was to normalize the image.

Afer that, the image was turned into a binary mask. Pixels were rounded to 1 if the intensity was greater than 0.5 and to 0 if less than 0.5.

##### A step-by-step example:

Get Spectrogram

In [ ]:
P, freqs, bins, im = plt.specgram(whale_audio, NFFT=256, Fs=whale_samplerate, noverlap=192)

Vertically Demean

In [ ]:
def slidingWindowV(P,inner=3,outer=64,maxM = 50,norm=True):
	Q = P.copy()
	m, n = Q.shape
	if norm:
		mval, sval = np.mean(Q[:maxM,:]), np.std(Q[:maxM,:])
		fact_ = 1.5
		Q[Q > mval + fact_*sval] = mval + fact_*sval
		Q[Q < mval - fact_*sval] = mval - fact_*sval
	wInner = np.ones(inner)
	wOuter = np.ones(outer)
	for i in range(n):
		Q[:,i] = Q[:,i] - (np.convolve(Q[:,i],wOuter,'same') - np.convolve(Q[:,i],wInner,'same'))/(outer - inner)
	return Q[:maxM,:]

m, n = P.shape
P = slidingWindowV(P,maxM=m)
plot_specgram(P, freqs, bins)

Normalize and Turn into Binary Mask

In [ ]:
tmpl = P.astype('Float32')
m_, s_ = np.mean(tmpl), np.std(tmpl)
min_ = tmpl.min()
tmpl[ tmpl < m_ + 0.5*s_] = min_ 
tmpl[ tmpl > min_ ] = 1
tmpl[tmpl < 0] = 0

plot_specgram(tmpl, freqs, bins)

Cut out a Rectangle

In [ ]:
x0=12
xn=26
y0=23
yn=50

tmpl = P[x0:xn,y0:yn].astype('Float32')
m_, s_ = np.mean(tmpl), np.std(tmpl)
min_ = tmpl.min()
tmpl[ tmpl < m_ + 0.5*s_] = min_ 
tmpl[ tmpl > min_ ] = 1
tmpl[tmpl < 0] = 0

Z = np.flipud(tmpl)
		
extent = x0, xn, y0, yn
figure()
im = imshow(Z, extent=extent)
axis('auto')
xlim([x0, xn])
ylim([y0, yn])
show()

#### Generating a Feature from Matching

When template matching is called on a source image and template, the result is the (x,y) coordinate with the maximum correlation to the template as well as the score it obtained. There are many different scoring formulas but the one used in this solution is the equation mentioned above. This process is repeated for every template. Therefore what they end up with is a list of how well an image correlates to a template, for every template.

### High Frequency Metrics

TODO

### Other Metrics

Other metrics used as features were the centroid, width, skew, and total variation of a time slice of frequencies. 

### Classification

After all of this feature generation, the results are sent to a Gradient Boosting Classifier.

Gradient Boosting is a machine learning tecnique for regression problems. It produces a prediction model in the form of an ensemble of weak prediction models. Gradient boosting predictors are built in a stage-wise fashion. Gradient boosting is used for classification by reducing the classification task to a regression problem with a suitable loss function.

### Results

The method outline here received a 0.981 ROC AUC score. It received an even higher score when temporal information in the file naming was used as a feature. Files in the dataset were numbered sequentially, and incidently were also chronologically ordered. What ended up happening was that clusters of whale calls would appear in proximity to each other temporaly, represented as filenames that were close to each other. By utilizing this information, the score jumped to 0.9838.

## Evaluation Criteria

### Area under the Receiver Operating Characteristic Curve

From Wikipedia - Receiver operating characteristic (ROC), or simply ROC curve, is a graphical plot which illustrates the performance of a binary classifier system as its discrimination threshold is varied. It is created by plotting the fraction of true positives out of the positives (TPR = true positive rate) vs. the fraction of false positives out of the negatives (FPR = false positive rate), at various threshold settings.

The AUC is the area of the region underneath this curve.

In other words, the ROC Curve is the sensitivity (true positives rate) plotted against false positives rate (1-specificity) as the discrimination threshold is varied.

True positive rate is also called sensitiviy, recall, or hit-rate.

$$
tprate=\frac{\text{Positives correctly classified}}{\text{Total positives}}
$$

The false positive rate (also called false alarm rate) of the
classifier is

$$
fprate=\frac{\text{Negatives incorrectly classified}}{\text{Total negatives}}
$$

#Convolutional Neural Network Approach

##Neural Networks

Neural Networks are models inspired by animal central nervous systems (in particular the brain) that are capable of machine learning and pattern recognition. The first generation of neural networks, called perceptrons, were introduced in 1957 by Frank Rosenblatt. They were deemed ineffective by the artificial intelligence community. Recently, neural networks have made a comeback as we now have more tools to properly train them.

##Convolutional Neural Network

Convolutional Neural Networks (CNN) are variants of Multi-layer Perceptrons. They are modeled after Hubel and Wisel's early work on the cat's visual cortex. They are designed to recognize visual patterns directly from pixel images with minimal preprocessing. They can recognize patterns with extreme variability, and with robustness to distortions and simple geometric transformations.

###Preparing images

A CNN take images as input, therefore it is natural that we pass it our mean spectrograms as input. We first scale down our mean spectrograms to 64x29 px images with 3 channels. 

###Storing

We store our data in HDF5 datasets using the H5PY library in Python. This allows us to store our data on disk, instead of RAM, which is good for large datasets. Pylearn2, requires data in the form of an object of type DenseDesignMatrix which provides special handlers for dealing with the dataset. A simple wrapper is created to wrap the HDF5 datasets as a DenseDesignMatrix.

###Hidden Layers

There are two hidden layers to the neural network. Both contain a convolution layer and a subsampling layer in the form of a single ConvRectifiedLinear layer. These layers make the neural network invariant to small changes such as translation or scale.

###Soft-Max Layer

The Soft-Max layer calculates the output of the neural network from the input of the layers above. Essentially, it performs logistic regression after applying weights to the input and gives out a prediction.

###Training the Network

Since the output of one layer depends on another, we make a full pass over the network and calculate the derivative of the error. We use stochastic gradient descent while backpropagating the derivative of the error to the previous layers to adjust the weights of each unit in the network.

###Putting it all together